# dAMN with M28 dataset

## Train

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import utils
import numpy as np
import tensorflow as tf
print('Physical GPUs:', tf.config.list_physical_devices('GPU'))

train_test_split = 'forecast' # 'forecast' or 'medium'
folder = './'
file_name = 'M28_OD_20'
media_file = folder+'data/'+'M28_media.csv'                 
od_file = folder+'data/'+file_name+'.csv'               
cobra_model_file = folder+'data/'+'iML1515_duplicated.xml'
biomass_rxn_id = 'BIOMASS_Ec_iML1515_core_75p37M'

# Hyperparameters
seed = 10
np.random.seed(seed=seed)
hidden_layers_lag = [50]
hidden_layers_flux = [500]
num_epochs = 1000
x_fold = 3      
batch_size = 10
patience = 100
N_iter = 3
run_name = f'{file_name}_{train_test_split}'

# Create model
model, train_array, train_dev, val_array, val_dev, val_ids = utils.create_model_train_val(
    media_file, od_file,
    cobra_model_file,
    biomass_rxn_id,
    x_fold=x_fold, 
    hidden_layers_lag=hidden_layers_lag, 
    hidden_layers_flux=hidden_layers_flux, 
    dropout_rate=0.2,
    loss_weight=[0.001, 1, 1, 1], 
    loss_decay=[0, 0.5, 0.5, 1], 
    verbose=True,
    train_test_split=train_test_split
)

# Saving for future testing and validation
np.savetxt(f'{folder}model/{run_name}_val_array.txt', val_array, fmt='%f')
np.savetxt(f'{folder}model/{run_name}_val_dev.txt', val_dev, fmt='%f')
np.savetxt(f'{folder}model/{run_name}_val_ids.txt', np.asarray(val_ids), fmt='%d')

# Train model
for i in range(N_iter):
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-3,
        decay_steps=280,
        decay_rate=0.9,
        staircase=True
    )
    (
        (losses_s_v_train, losses_neg_v_train, losses_c_train, losses_drop_c_train),
        (losses_s_v_val, losses_neg_v_val, losses_c_val, losses_drop_c_val)
    ) = utils.train_model(
        model, train_array, val_array=val_array,
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        num_epochs=num_epochs, batch_size=batch_size, patience=patience,
        verbose=True,
        train_test_split=train_test_split,
        x_fold=x_fold
    )

    model_name = f'{folder}model/{run_name}_{str(i)}'
    model.save_model(model_name=model_name, verbose=True)

    utils.plot_loss('Training S_v', losses_s_v_train, num_epochs, save="./figure")
    utils.plot_loss('Training Neg_v', losses_neg_v_train, num_epochs, save="./figure")
    utils.plot_loss('Training C', losses_c_train, num_epochs, save="./figure")
    utils.plot_loss('Training Drop_c', losses_drop_c_train, num_epochs, save="./figure")

    if x_fold > 1:
        utils.plot_loss('Validation S_v', losses_s_v_val, num_epochs, save="./figure")
        utils.plot_loss('Validation Neg_v', losses_neg_v_val, num_epochs, save="./figure")
        utils.plot_loss('Validation C', losses_c_val, num_epochs, save="./figure")
        utils.plot_loss('Validation Drop_c', losses_drop_c_val, num_epochs, save="./figure")


## Test

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import sklearn
import utils
import numpy as np
import tensorflow as tf
from sklearn.metrics import r2_score

metabolite_ids = [
'glc__D_e', 'xyl__D_e', 'succ_e', 'ala__L_e', 'arg__L_e', 'asn__L_e', 'asp__L_e',
'cys__L_e', 'glu__L_e', 'gln__L_e', 'gly_e', 'his__L_e', 'ile__L_e', 'leu__L_e',
'lys__L_e', 'met__L_e', 'phe__L_e', 'pro__L_e', 'ser__L_e', 'thr__L_e', 'trp__L_e',
'tyr__L_e', 'val__L_e', 'ade_e', 'gua_e', 'csn_e', 'ura_e', 'thymd_e', 'BIOMASS'
]

train_test_split = 'forecast' # 'forecast' or 'medium'
folder = './'
file_name = 'M28_OD_20'
N_iter = 3
run_name = f'{file_name}_{train_test_split}'
OD = True # when True biomass concentration transformed in OD
plot =  'growth' # 'growth' or 'substrate'

# Load
val_array = np.loadtxt(f'{folder}model/{run_name}_val_array.txt', dtype=float)
val_dev = np.loadtxt(f'{folder}model/{run_name}_val_dev.txt', dtype=float)
val_ids = np.loadtxt(f'{folder}model/{run_name}_val_ids.txt', dtype=int)
if val_array is None:
    raise ValueError(f'Validation file not found: {folder}model/{run_name}_val_array.txt')

# Predict
Pred, Ref, Pred_bio, Ref_bio = {}, {}, {}, {}
for i in range(N_iter):
    model_name = f'{folder}model/{run_name}_{str(i)}'
    model = utils.MetabolicModel.load_model(model_name=model_name, verbose=False)
    model.metabolite_ids = metabolite_ids if len(model.metabolite_ids) == 0 else model.metabolite_ids
    pred, ref = utils.predict_on_val_data(model, val_array, verbose=False) # 1, 86, 157
    pred, ref = np.asarray(pred), np.asarray(ref)
    Pred[i], Ref[i] = pred, ref
Pred , Ref = np.asarray(list(Pred.values())), np.asarray(list(Ref.values()))
R2 = utils.r2_growth_curve(Pred, Ref, OD=OD)

print(f'Model: {run_name}  R2 = {np.mean(R2):.2f}±{np.std(R2):.2f} Median = {np.median(R2):.2f}')
title = f"R2 Histogram {train_test_split}"
utils.plot_similarity_distribution(title, R2, save="./figure")

# Plot
if plot == 'growth':
    utils.plot_predicted_reference_growth_curve(
    times=model.times,
    Pred=Pred, Ref=Ref, val_dev=val_dev,
    OD=OD,R2=R2,
    train_time_steps=model.train_time_steps if hasattr(model, "train_time_steps") else 0,
    experiment_ids=list(val_ids),
    run_name=run_name,
    train_test_split=train_test_split,
    save="./figure"
    )
elif plot == 'substrate':
    utils.plot_predicted_biomass_and_substrate(
    model.times, Pred,
    experiment_ids=list(val_ids),
    metabolite_ids=list(model.metabolite_ids),
    run_name=run_name,
    train_test_split=train_test_split,
    save="./figure"
    )


## Parameter search

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import utils
import numpy as np
import tensorflow as tf
import random
from itertools import product
import json
from sklearn.metrics import r2_score

train_test_split = 'forecast' # 'forecast' or 'medium'
folder = './'
media_file = folder+'data/'+'M28_media.csv'
od_file = folder+'data/'+'M28_OD_20.csv'
cobra_model_file = folder+'data/'+'iML1515_duplicated.xml'
biomass_rxn_id = 'BIOMASS_Ec_iML1515_core_75p37M'
hidden_layers_lag = [50]
hidden_layers_flux = [500]

# Parameter search spaces
l = [0.0001, 0.001, 0.01, 0.1, 1.0, 1, 2]
k = [0.0, 0.25, 0.5, 0.75, 1.0]
l1, l2, l3, l4 = l.copy(), l.copy(), l.copy(), l.copy() 
k1, k2, k3, k4 = k.copy(), k.copy(), k.copy(), k.copy()
N_search = 25
param_grid = list(product(l1, k2, k3, k4))
random.seed(1)
param_samples = random.sample(param_grid, min(N_search, len(param_grid)))
short_num_epochs = 500
batch_size = 10
patience = 30
x_fold = 3
results = []
l1_, l2_, l3_, l4_ = 0.001, 1, 1, 1
k1_, k2_, k3_, k4_ = 0, 0.5, 0.5, 1

for idx, (l1_, k2_, k3_, k4_) in enumerate(param_samples):
    # Create train and val sets
    model, train_array, train_dev, val_array, val_dev, val_ids = utils.create_model_train_val(
        media_file, od_file, cobra_model_file, biomass_rxn_id,
        x_fold=x_fold,
        hidden_layers_lag=hidden_layers_lag,
        hidden_layers_flux=hidden_layers_flux,
        dropout_rate=0.2,
        loss_weight =[l1_, l2_, l3_, l4_],
        loss_decay  =[k1_, k2_, k3_, k4_],
        verbose=False,
        train_test_split=train_test_split
    )
    
    # Train
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-3, decay_steps=280, decay_rate=0.9, staircase=True
    )
    (
        (losses_s_v_train, losses_neg_v_train, losses_c_train, losses_drop_c_train),
        (losses_s_v_val, losses_neg_v_val, losses_c_val, losses_drop_c_val)
    ) = utils.train_model(
        model, train_array, val_array=val_array,
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        num_epochs=short_num_epochs, batch_size=batch_size, patience=patience,
        verbose=False,
        train_test_split=train_test_split,
        x_fold=x_fold
    )
    
    # Predict
    pred, ref = utils.predict_on_val_data(model, val_array, verbose=False)
    pred, ref = utils.concentration_to_OD(pred),  utils.concentration_to_OD(ref) 
    pred = np.expand_dims(pred, axis=0)
    ref = np.expand_dims(ref, axis=0)
    R2 = utils.r2_growth_curve(pred, ref, OD=True)
    R2_mean, R2_std = np.mean(R2), np.std(R2)
    
    print(f'*** Testing λ={[l1_, l2_, l3_, l4_]}, k={[k1_, k2_, k3_, k4_]} ({idx+1}/{len(param_samples)}) '\
          f'R2: {R2_mean:.3f} ± {R2_std:.3f}')
    
    # Record
    results.append({
        "l1": l1_,
        "l2": l2_,
        "l3": l3_,
        "l4": l4_,
        "k1": k1_,
        "k2": k2_,
        "k3": k3_,
        "k4": k4_,
        "R2_mean": R2_mean,
        "R2_std":  R2_std 
    })

results = sorted(results, key=lambda x: x["R2_mean"], reverse=True)
print("\nFull Table: Hyperparameter Search Results")
print("l1\tl2\tl3\tl4\tk1\tk2\tk3\tk4\tR2-mean\tR2-std")
for res in results:
    print(f"{res['l1']}\t{res['l2']}\t{res['l3']}\t{res['l4']}\t{res['k1']}\t{res['k2']}\t{res['k3']}\t{res['k4']}\t{res['R2_mean']:.3f}\t{res['R2_std']:.3f}")
